## config

In [1]:
import os
import glob
import re
import pandas as pd

In [2]:
## config the path where multiple files contained.
## NEED modify every time it runs !!!

path = "/home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope"
homobed_or_blasp = "homoblastp"
'''
select list is "homobed","homoblastp","bedblastp"
"homobed" means homobed file as input only
"homoblastp" means homoblastp file as input only
"bedblastp" means homobed and homoblastp files both as input
'''
identity = 65

In [3]:
## config input files under the given path.
## No NEED modify every time it runs !!!

path_mhcI = path + "/02.protein_antigen_prediction/parsed_res_MHCI"
path_mhcII = path + "/02.protein_antigen_prediction/parsed_res_MHCII"
trans_memb = path + "/04.TransMembrane.DeepTMHMM/TransMembrane.bed"
enzymedigest = path + "/05.EnzymeDigest/protein.merge.enzymedigest"

if homobed_or_blasp=="homobed":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
elif homobed_or_blasp== "homoblastp":
    homo_blastp = path + "/03.homologous/peptide_netMHC.blastp"
elif homobed_or_blasp== "bedblastp":
    homo_bed = path + "/03.homologous/homo_peptide.bed"
    homo_blastp = path + "/03.homologous/peptide_netMHC.blastp"

outpath = path + "/06.Deliverables/EpitopePresent"
outpath1 = path + "/06.Deliverables/EpitopePresent/MHCI"
outpath2 = path + "/06.Deliverables/EpitopePresent/MHCII"
try:
    os.mkdir(outpath)
except:
    pass
try:
    os.mkdir(outpath1)
except:
    pass
try:
    os.mkdir(outpath2)
except:
    pass

#### Tools

In [4]:
import difflib
def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return(size,s1[pos_a:pos_a+size])
s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

(39, 'GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC')


In [5]:
def get_homodict_bed(Homo_parsDict,seqid,pos1,pos2):
    checklist = [homo_f==0 for homo_f in Homo_parsDict[seqid][pos1:pos2]]
    if  sum(checklist)/(pos2-pos1) >= 0.05:
        HomoEx = "N"
        Homo_flag = ""
    else:
        HomoEx = "Y"
        flagtemp_list = []
        for flag in Homo_parsDict[seqid][pos1:pos2]:
            if flag !=0:
                flagtemp_list += flag
        Homo_flag = ";".join(list(dict.fromkeys(flagtemp_list)))
    return(HomoEx,Homo_flag)

def get_homodict_blastp(Homoblastp_parsDict,seqid,peptide):
    try:
        homolist = Homoblastp_parsDict[seqid][peptide]
        if len(homolist) == 0:
            Homo_flag = ""
            HomoEx = "N"
        else:
            Homo_flag = ';'.join(homolist)
            HomoEx = "Y"
    except:
        Homo_flag = ""
        HomoEx = "N"
    return(HomoEx,Homo_flag)
    

## Step1: Parser the TransMemb.bed

In [6]:
## Parser the TransMemb.bed
TM_parsDict={}
flagDict={"o":"outside","T":"TMhelix","i":"inside","s":"signal"}
'''
    parser the tranmembe.bed file
    TM_flag=outside,TMhelix,inside
    outside:o
    TMhelix:t
    inside:i
'''
with open(trans_memb,"r") as tmf:
    for line in tmf.readlines():
        if line.startswith("sp|"):
            seqid = line.split("\t")[0].split("|")[1]
            if seqid not in TM_parsDict.keys():
                TM_parsDict[seqid] = ""
            pos1 = int(line.split("\t")[1])
            pos2 = int(line.split("\t")[2])
            TM_flag = line.split("\t")[3][0]
            TM_parsDict[seqid] += TM_flag*(1+pos2-pos1)

## Step2: Parser the Homologous_peptide.bed

In [7]:
## Parser the Homologous_peptide.bed
if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
    Homobed_parsDict={}
    with open(homo_bed,"r") as hmf:
        for line in hmf.readlines():
            if line.startswith("sp|"):
                #print(line)
                seqid = line.split("\t")[0].split("|")[1]
                if seqid not in Homobed_parsDict.keys():
                    length = len(TM_parsDict[seqid])
                    Homobed_parsDict[seqid] = [0]*length
                pos1 = int(line.split("\t")[1])-1
                pos2 = int(line.split("\t")[2])-1
                homoid = line.split("\t")[3]
                for pos in range(pos1,pos2):
                    #print(pos)
                    if Homobed_parsDict[seqid][pos] == 0:
                        Homobed_parsDict[seqid][pos] = [homoid]
                    else:
                        Homobed_parsDict[seqid][pos].append(homoid)

In [8]:
## Parser the peptide_netMHC.blastp
if (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
    Homoblastp_parsDict={}
    headname= ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
    blastdf = pd.read_csv(homo_blastp,sep="\t",names=headname)
    blastdf[["protein","peptide"]] = blastdf.qseqid.str.split(":",expand=True)
    #print(blastdf.head(10))
    blastdf["proteinUnipID"] = blastdf.protein.str.split("_").str[1]
    #print(blastdf.proteinUnipID.unique().tolist())
    proteinlist = blastdf.proteinUnipID.unique().tolist()
    selfUniproList = []
    for proteinid in proteinlist:
        # calculate the self 
        qblastdf = blastdf[blastdf["proteinUnipID"]==proteinid]
        homoRefseries = qblastdf.sseqid.value_counts()
        # calculate the query number (or bin numbers)
        querynum = len(qblastdf.qseqid.unique())
        # find those reference id which is the query ploymer 
        selfUniproID = homoRefseries[homoRefseries>querynum*0.9].index.tolist()
        selfUniproList += selfUniproID
        print(selfUniproID)
    print(selfUniproList)
 
    # remove those aligned lines which reference is it's self
    noSelfBlastdf = blastdf[~blastdf.sseqid.isin(selfUniproList)]
    # select those aligned lines which pident bigger than cutoff
    noSelfHomodf = noSelfBlastdf[noSelfBlastdf.pident > identity]
    
    noSelfHomodf2 = noSelfHomodf[["proteinUnipID","peptide","sseqid"]]
    #print(noSelfHomodf2.head(20))
    for proteinid in proteinlist:
        print(proteinid)
        noSelfHomodf2_p = noSelfHomodf2[noSelfHomodf2["proteinUnipID"]==proteinid]
        peptidelist = noSelfHomodf2_p.peptide.unique().tolist()
        peptidesseqiddict = {}
        for peptide in peptidelist:
            sseqidlist = noSelfHomodf2_p[noSelfHomodf2_p["peptide"]==peptide].sseqid.tolist()
            peptidesseqiddict[peptide] = sseqidlist
            Homoblastp_parsDict[proteinid] = peptidesseqiddict
    #print(Homoblastp_parsDict)
  

['O75388']
['P04053', 'P04053-2']
['P10242', 'P10242-4', 'P10242-2', 'P10242-7', 'P10242-8', 'P10242-9']
['P15814']
['P19544-4', 'P19544', 'P19544-7', 'P19544-2', 'P19544-8', 'P19544-3']
['P30939']
['P40238']
['Q00534']
['Q01851']
['Q5HYN5', 'Q8NHU0', 'P0DMV0', 'P0DMU7', 'P0DMU8', 'P0DMV2', 'P0DMV1', 'Q5DJT8']
['Q6ZV50']
['Q8IYM0']
['Q96A99']
['O75388', 'P04053', 'P04053-2', 'P10242', 'P10242-4', 'P10242-2', 'P10242-7', 'P10242-8', 'P10242-9', 'P15814', 'P19544-4', 'P19544', 'P19544-7', 'P19544-2', 'P19544-8', 'P19544-3', 'P30939', 'P40238', 'Q00534', 'Q01851', 'Q5HYN5', 'Q8NHU0', 'P0DMV0', 'P0DMU7', 'P0DMU8', 'P0DMV2', 'P0DMV1', 'Q5DJT8', 'Q6ZV50', 'Q8IYM0', 'Q96A99']
O75388
P04053
P10242
P15814
P19544
P30939
P40238
Q00534
Q01851
Q5HYN5
Q6ZV50
Q8IYM0
Q96A99


In [9]:
#print(Homoblastp_parsDict["Q96A99"])

{'AALAGGFVQGA': ['Q96A99-2'], 'AALAGGFVQG': ['Q96A99-2'], 'AALEGRLPVA': ['Q96A99-2'], 'AALEGRLPVAH': ['Q96A99-2', 'P51784'], 'AALGPALVPT': ['Q96A99-2'], 'AALGPALVPTP': ['Q96A99-2'], 'AASEHRGPP': ['Q96A99-2'], 'AASEHRGPPQ': ['Q96A99-2'], 'AASEHRGPPQD': ['Q96A99-2'], 'AEESQAVAQAV': ['Q96A99-2'], 'AFRELPLQL': ['Q96A99-2'], 'AFRELPLQLL': ['Q96A99-2'], 'AFRELPLQLLL': ['Q96A99-2'], 'AFVGSMSGLA': ['Q96A99-2'], 'AFVGSMSGL': ['Q96A99-2'], 'AFVGSMSGLAI': ['Q96A99-2'], 'AGGFVQGAN': ['Q96A99-2'], 'AGGFVQGANC': ['Q96A99-2'], 'AGGFVQGANCT': ['Q96A99-2'], 'AHKAQRDALQ': ['Q96A99-2'], 'AHKAQRDALQD': ['Q96A99-2'], 'AHPGTAALG': ['Q96A99-2'], 'AHPGTAALGPA': ['Q96A99-2'], 'AHPGTAALGP': ['Q96A99-2'], 'AIGKEFPTGA': ['Q96A99-2'], 'AIGKEFPTGAI': ['Q96A99-2'], 'AIGKEFPTG': ['Q96A99-2'], 'AILTLANAALA': ['Q96A99-2'], 'AILTLANAAL': ['Q96A99-2'], 'AIWDRALV': ['Q96A99-2'], 'AIWDRALVP': ['Q96A99-2'], 'AIWDRALVPG': ['Q96A99-2', 'Q15818'], 'AIWDRALVPGE': ['Q96A99-2', 'Q15818', 'Q6QNK2-4', 'Q6QNK2'], 'ALAGGFVQGA': ['Q96

## Step3: Parser the enzymedigest file

In [10]:
## Parser the enzymedigest file
EnzDigeDict={}
with open(enzymedigest,"r") as enzymef:
    for line in enzymef.readlines():
        refseq = re.split(',"\[',line)[0].split(",")[-1]
        refid = re.split(',"\[',line)[0].split("|")[1]
        seqlist = re.split(',"\[',line)[1].strip().replace(']"','').replace("'","").split(",")
        EnzDigeDict[refid]=seqlist
#print(EnzDigeDict)

In [11]:
ls /home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/06.EnzymeDisgest/protein.merge.enzymedigest

ls: cannot access '/home/jovyan/work/02.ResDev/12.TAA_epitope_human/05.AML_TAA_Epitope/06.EnzymeDisgest/protein.merge.enzymedigest': No such file or directory


In [12]:
## Parser the netMHC.csv
mergeoutfile =  open(outpath+"/MHCI_Epitopeonly_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcI + "/*.netMHC.csv"):
    filenum +=1
    filename= os.path.basename(file)
    outfile = open(outpath1+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            """
            itemlist is the line content of netMHC result
            """
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                seqid = itemlist[10].split("_")[1]
                midpos = int((pos1+pos2)/2)
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"/"+str(len(peptide))+'"'
                Cut_seq = overseq0
                
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,Cut_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

In [13]:
## Parser the netMHC.csv
mergeoutfile2 =  open(outpath+"/MHCII_Epitope_only_SB_WB.csv","w")
filenum = 0
for file in glob.glob(path_mhcII + "/*.netMHC.csv"):
    filenum += 1
    filename= os.path.basename(file)
    outfile = open(outpath2+"/"+filename,"w")
    with open(file,"r") as eptf:
        for line in eptf.readlines():
            itemlist = line.rstrip().split(",")
            if line.startswith("Pos"):
                appendlist = ["TransMemb","InCutmerRate","InCutmerRegion","HomoExsit","HomoId"]
                outfile.write(",".join(itemlist+appendlist)+"\n")
                if filenum == 1:
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
            else:
                pos1 = int(itemlist[0])-1
                peptide = itemlist[2]
                pos2 = pos1 + len(peptide)
                seqid = itemlist[6].split("|")[1]
                midpos = int((pos1+pos2)/2)
                
                ## get Transmembrane info
                TM_f = TM_parsDict[seqid][midpos]
                TM_flag = flagDict[TM_f]
                
                ## get enzymedigest info
                size0 = 0
                overseq0 = ""
                for seq in EnzDigeDict[seqid]:
                    size,overseq = get_overlap(peptide,seq)
                    if size > size0:
                        size0 = size
                        overseq0 = overseq
                Cut_flag = '"'+str(size0)+"/"+str(len(peptide))+'"'
                Cut_seq = overseq0
                
                ## get homolgous info
                if (homobed_or_blasp == "homobed") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_bed(Homo_parsDict,seqid,pos1,pos2)
                elif (homobed_or_blasp == "homoblastp") or (homobed_or_blasp == "bedblastp"):
                    HomoEx,Homo_flag = get_homodict_blastp(Homoblastp_parsDict,seqid,peptide)
                        
                ##  output the results with filtering the "SB" lines
                if (itemlist[-1] == "SB") or (itemlist[-1] == "WB"):
                    appendlist = [TM_flag,Cut_flag,Cut_seq,HomoEx,Homo_flag]
                    mergeoutfile2.write(",".join(itemlist+appendlist)+"\n")
                else:
                    appendlist = ["",TM_flag,HomoEx,Homo_flag]
                outfile.write(",".join(itemlist+appendlist)+"\n")
    outfile.close()

In [14]:
mylist = ["a", "b", "a", "c", "c"]
mylist = list(dict.fromkeys(mylist))
print(mylist)

['a', 'b', 'c']


In [15]:
";".join(list(dict.fromkeys(mylist)))

'a;b;c'

In [16]:
fasta = 'CGTAGCGGCTGAGTGCGCGGATAGCGCGTA'
target = 'CGGCTGAGT'

start = fasta.find(target)+1
end = start + len(target) -1

print(start, end)

6 14


In [17]:
import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

s1 = "CGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC"
s2 = "GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGC"

print(get_overlap(s1, s2)) # GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC

GGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTC
